In [1]:
from torch_geometric.loader import DataLoader,NodeLoader
from torch_geometric.utils import to_scipy_sparse_matrix
from torch import nn
from torch.nn import functional as F
import torch as th
from tqdm import tqdm 

In [3]:
class graphattention_layer(nn.Module):
    def __init__(self,input_size,output_size,adjM):
        '''WkH_{i-1} is of dimension : CurrentNodeShape x N'''  
        self.inpshape = input_size
        self.opshape = output_size
        self.A = adjM
        super(graphattention_layer,self).__init__()
        self.vks = nn.Linear(in_features=output_size,out_features=  1)
        self.vkr = nn.Linear(in_features=output_size,out_features= 1)
        self.W =  nn.Linear(in_features=input_size,out_features=output_size) 
    def forward(self, H_k,A):
        '''H_k represents the previous layer's graph representation'''
        '''So i have to account for subgraph forward passes,?'''
        if(A is None):
            M_s = self.A * self.vks(F.relu(self.W(H_k))).T
            M_r = (self.A * self.vkr(F.relu(self.W(H_k))).T).T
            Attention = F.softmax(F.sigmoid(M_s+M_r))
            H_new = Attention@F.relu(self.W(H_k))
            return H_new
        else:
            M_s = A * self.vks(F.relu(self.W(H_k))).T
            M_r = (A * self.vkr(F.relu(self.W(H_k))).T).T
            Attention = F.softmax(F.sigmoid(M_s+M_r))
            H_new = Attention@F.relu(self.W(H_k))
            return H_new  

In [4]:
class encoder(nn.Module):
    def __init__(self,adjM,input_embeddings):
        super(encoder,self).__init__( )
        ''' 
        remember that in pytorch, your input_size is the last dimension of your input
        So when my input is F*N, input_size = F
        also a row in my matrix corresponds to a cell's representation
        '''
       
        self.layer1 = graphattention_layer(input_size=input_embeddings
                                           ,output_size=512
                                           ,adjM=adjM)
        self.layer2 = graphattention_layer(input_size=512
                                           ,output_size=256
                                           ,adjM=adjM)
        self.layer3 = graphattention_layer(input_size=256
                                           ,output_size=64
                                           ,adjM=adjM)
    def forward(self, X,A):
        '''
        X here is the node embeddings, its of shape (N*embedding_size)
        I'm gonna tranpose it once in the start, and then at the end.
        H3 is of size N*64
        I'm gonna transpose it back to 64*N
        '''
        H1 = self.layer1(X,A)
        H2 = self.layer2(H1,A)
        H3 = self.layer3(H2,A)
        return H3

In [5]:
    
class decoder(nn.Module):
    def __init__(self,adjM,reconstruction_embedding):
        super(decoder,self).__init__()
        self.layer1 = graphattention_layer(input_size=64,
                                           output_size=256,adjM=adjM)
        self.layer2 = graphattention_layer(input_size=256,
                                           output_size=512,adjM=adjM)
        self.layer3 = graphattention_layer(input_size=512,
                                           output_size=reconstruction_embedding,adjM=adjM)
    def forward(self, H,A):
        '''
        H here is the encoder's output
        I'm gonna stack the gene embeddings to the H matrix
        Encoder should have returned a 64*N matrix
        Gene embeddings should be of dimension 64* num_nodes , which was 647 for the first run.
        So we're concatenating a 64*647 matrix to a 64*N
        '''
        # Now its a (N)*64 matrix
        H1 = self.layer1(H,A)
        H2 = self.layer2(H1,A)
        H3= self.layer3(H2,A)
        ''' H3 would be of size N*N'''
        return H3


In [6]:
class GAE(nn.Module):
    def __init__(self,adjM,innode_embedding,edge_index):
        super(GAE,self).__init__()
        self.adjM = adjM
        self.encoder = encoder(adjM,innode_embedding)
        self.decoder = decoder(adjM,innode_embedding)
        self.edge_index = edge_index
        self.mse = nn.MSELoss()
    def forward(self,X,A=None):
        encoded = self.encoder(X,A)
        decoded = self.decoder(encoded,A)
        if(A is None):
            hi_hj = (encoded@encoded.T)
            masked_product = self.adjM*hi_hj
            sig = nn.Sigmoid()(masked_product)
            epsilon = 1e-9
            self.edge_loss = -th.sum(self.adjM*(th.log(sig)))  
            # source,target = self.edge_index
            # source_embeddings = encoded[source]
            # target_embeddings = encoded[target]
            # hi_hj = source_embeddings@(target_embeddings.T)
            # self.edge_loss = th.sum(nn.Softmax()(hi_hj))
        else:
            self.edge_loss =0 
            pass
        
        return decoded

In [7]:
from torch_geometric.datasets import Planetoid,PPI

#dataset = PPI(root='./',split='train')
train_dataset = Planetoid(root='./',name='Cora')
#train_dataset = PPI(root='./', split='train')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [12]:
adjM = th.tensor(to_scipy_sparse_matrix(train_dataset.edge_index).toarray())
gae_model = GAE(adjM,train_dataset.x.shape[1],train_dataset.edge_index)

In [17]:
fake_features = (th.randn(train_dataset[0].x.shape[0],train_dataset[0].x.shape[1]))*5 + 100

In [19]:
'''Training Loop:'''
num_epochs = 20
criterion = nn.MSELoss()
optim = th.optim.Adam(gae_model.parameters())
for epoch in range(num_epochs):
    gae_model.train()
    epoch_loss = 0
    optim.zero_grad()
    decoded = gae_model(train_dataset[0].x)
    #print(th.sum(gae_model.encoder(train_dataset[0].x,None)))
    loss = criterion(decoded,train_dataset[0].x)
    loss = loss + 0.5*(gae_model.edge_loss)
    loss.backward(retain_graph=True)
    optim.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")
    # with tqdm(total=len(loader),
    #      desc=f"Epoch{(epoch+1)/num_epochs}") as pbar:
    #     for batch in loader:
        # optim.zero_grad()
        # decoded = gae_model(batch.x)
        # loss = criterion(decoded,batch.x)
        # #loss+= -0.5*(gae_model.edge_loss)
        # loss.backward()
        # optim.step()
        # pbar.update(1)
        # pbar.set_postfix(loss=loss.item())
    # print(f"Epoch {epoch+1} completed.")

C:\Users\saira\AppData\Local\Temp\ipykernel_7868\1288872741.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  Attention = F.softmax(F.sigmoid(M_s+M_r))


Epoch 1/20, Loss: 0.5732
Epoch 2/20, Loss: 0.0125
Epoch 3/20, Loss: 0.0125
Epoch 4/20, Loss: 0.0124
Epoch 5/20, Loss: 0.0124
Epoch 6/20, Loss: 0.0124
Epoch 7/20, Loss: 0.0124
Epoch 8/20, Loss: 0.0123
Epoch 9/20, Loss: 0.0123
Epoch 10/20, Loss: 0.0123
Epoch 11/20, Loss: 0.0123
Epoch 12/20, Loss: 0.0123
Epoch 13/20, Loss: 0.0123
Epoch 14/20, Loss: 0.0123
Epoch 15/20, Loss: 0.0123
Epoch 16/20, Loss: 0.0123
Epoch 17/20, Loss: 0.0123
Epoch 18/20, Loss: 0.0123
Epoch 19/20, Loss: 0.0123
Epoch 20/20, Loss: 0.0123


In [16]:
gae_model.eval()
print(set(gae_model(train_dataset.x)))

C:\Users\saira\AppData\Local\Temp\ipykernel_7868\1288872741.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  Attention = F.softmax(F.sigmoid(M_s+M_r))


{tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0000, 0.0518,  ..., 0.0000, 0.0030, 0.0000],
       grad_fn=<UnbindBackward0>), tensor([0.0000, 0.0